In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import re
from HelperFunctions import *
import io
%reload_ext Cython

In [ ]:
%%cython
from libc.math cimport isnan

def sortOutIdle(series, transit_interval):
    #     display(series)
    count = 0
    last_val = 0
    last_t = 0
    last_i = -1
    to_fill = False
    for i, (t, val) in enumerate(series.iteritems()):
        if isnan(val):
            to_fill = True
        else:
            if to_fill == True:
                if t - last_t < transit_interval:
                    series.iloc[last_i + 1 : i] = last_val
            to_fill = False
            last_val = val
            last_t = t
            last_i = i

    return series

In [ ]:
def getPtOccupancies(path, transit_interval=540):
    file_data = open(path)
    data = ""
#     count = 0
    for x in file_data:
        if x.startswith("time"):
            match = re.search(
                "^time.*\t(\d+\.\d+) veh.*(tr_\d+_\d+).*Passenger.*?(\d+).*", x
            )
            if match:
                #                 print(match.group(0))
                data += (
                    match.group(1) + ";" + match.group(2) + ";" + match.group(3) + "\n"
                )
#                 count += 1
#                 if count > 10000:
#                     break
    file_data.close()

    occupancies = pd.read_csv(
        io.StringIO(data), names=["time", "transporter", "passengers"], sep=";"
    )
    occupancies = occupancies.sort_values(by=["transporter", "time"])

    occupancies = occupancies.pivot_table(
        index="time", columns="transporter", values="passengers", aggfunc="last"
    )
    #     new_first_col = pd.DataFrame([[0] * len(occupancies.columns)], columns=occupancies.columns)
    #     occupancies = new_first_col.append(occupancies).fillna(method='ffill')
    occupancies = occupancies.apply(lambda x: sortOutIdle(x, transit_interval))

#     with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#         display(occupancies)
    average_occupancies = occupancies.sum(axis=1)/occupancies.count(axis=1)
    return average_occupancies

In [ ]:
# def test_getPtOccupancies():
path = "/home/helge/Programs/matsim/matsim_results/OwnIntermodalAccEgr/OptDrtCount/bim_30minTransit_wDiag/180/ITERS/it.0/0.occupancyAnalysis.txt"
path2 = "/home/helge/Programs/matsim/matsim_results/OwnIntermodalAccEgr/OptDrtCount/bim_5minTransit_wDiag/180/ITERS/it.0/0.occupancyAnalysis.txt"
av_occ_30min = getPtOccupancies(path, 540)
av_occ_5min = getPtOccupancies(path2, 540)

In [ ]:
fig, (ax1, ax2) = plt.subplots(2,1)

av_occ_30min.plot(ax=ax1)
av_occ_5min.plot(ax=ax2)
ax2.set_xlabel('time (s)')
ax2.set_ylabel('Pt Occupancy')
ax1.set_ylabel('Pt Occupancy')
# plt.savefig('Pt_Occupancy_180vehicles', dpi=200)
plt.show()